In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
#import mplcursors
from sklearn.neighbors import NearestNeighbors

bt = pd.read_csv('data_d_kor.csv', low_memory=False)
NaN_processing_bt = bt.fillna(0) # nan 0으로 처리
indexing_bt = NaN_processing_bt.set_index('date') #인덱스 날짜로 변경
print(indexing_bt.head())

              gold  commodity  k10bond   kospi
date                                          
2000-01-03  0.0000    -0.0016      0.0  0.0000
2000-01-04 -0.0201    -0.0096      0.0  0.0301
2000-01-05 -0.0057    -0.0096      0.0 -0.0687
2000-01-06  0.0011    -0.0005      0.0 -0.0259
2000-01-07  0.0018    -0.0098      0.0 -0.0126


In [2]:
def Select_Date(Start, End, Dataframe) :   # 구간 데이터 모으는 함수
    """
    gold = gold.loc['{}'.format(Start):'{}'.format(End), 'gold']
    k10bond = k10bond.loc['{}'.format(Start):'{}'.format(End),'k10bond']
    kospi= kospi.loc['{}'.format(Start):'{}'.format(End) ,'kospi']
    commodity= commodity.loc['{}'.format(Start):'{}'.format(End), 'commodity']
    """
    total = indexing_bt.loc['{}'.format(Start):'{}'.format(End)]

    Dataframe = pd.concat([Dataframe,total]) # 리스트형식.. tqtqtqtqtqtq
    return Dataframe


def Get_randomized_weights():

    a = random.randrange(0, 21)
    b = random.randrange(0, 21-a)
    c = random.randrange(0, 21-a-b)
    d = random.randrange(0, 21-a-b-c)
    e = 20-a-b-c-d

    array = np.array([a,b,c,d,e])*5
    return array

def Analysis(Dataframe):
    """
    print(Dataframe.idxmax())

    print('----------Return Max------------')
    print(Dataframe.loc[Dataframe.idxmax()[0]])
    print('----------Sharpe Max------------')
    print(Dataframe.loc[Dataframe.idxmax()[2]])
    print('----------MDD Min------------')
    print(Dataframe.loc[Dataframe.idxmax()[1]])
    print('----------Volatility Min------------')
    print(Dataframe.loc[Dataframe.idxmin()[3]])
    """

    KNN(Dataframe)


def KNN(Dataframe):


    M = 10 # 이웃 몇개 채취할거임?

    x = Dataframe.iloc[:, 2]  # 샤프
    x = x.values.reshape(len(Dataframe),1) # fit 시키기 위해 변형

    neighbor = NearestNeighbors(n_neighbors=M, radius=0.4)
    neighbor.fit(x)


    nparray1 = neighbor.kneighbors(Dataframe.iloc[Dataframe.idxmax()[2], 2].reshape(-1, 1), M, return_distance=False)
    #print(Dataframe.loc[Dataframe.idxmax()[2]])
    #print(Dataframe.iloc[nparray1[0]])

    kospi = 0
    k10bond = 0
    k5bond = 0
    gold = 0
    commodity = 0


    for i in range(0, M):

        kospi += Dataframe.loc[nparray1[0][i], 'kospi']
        k10bond += Dataframe.loc[nparray1[0][i], 'k10bond']
        k5bond += Dataframe.loc[nparray1[0][i], 'k5bond']
        gold += Dataframe.loc[nparray1[0][i], 'gold']
        commodity += Dataframe.loc[nparray1[0][i], 'commodity']

    print("\n높은 Sharpe 지수를 가진 상위 {}개의 포트폴리오를 샘플링하여 평균낸 결과 \n코스피 지수 추종 주식 {}%, 한국채 10년 {}%, 한국채 5년 {}%, 금 {}%, 원자재 {}% 포트폴리오가 제안되었습니다. ".format(M, kospi/M, k10bond/M, k5bond/M, gold/M, commodity/M))

    print("\n가장 높은 Sharpe 지수를 보인 포트폴리오는 아래와 같습니다. ")
    print(Dataframe.loc[Dataframe.idxmax()[2]])



def Scatter(Dataframe):

    # scatter
    plt.subplots(figsize=[10, 10])
    scatter1 = plt.scatter(Dataframe['Volatility'], Dataframe['Return %'], marker='o', s=10, alpha=0.5) #알파는 투명도


    cursor1 = mplcursors.cursor(scatter1, hover=True)
    cursor1.connect("add", lambda sel: sel.annotation.set_text('Portfolio \n Index : {}\n Return : {}%\n Volatility : {}\n Sharpe : {}\n MDD : {}%\n kospi : {}\n k10bond : {}\n k5bond : {}\n gold : {}\n commodity : {}'.format(sel.target.index, sel.target[1], sel.target[0], Dataframe.iloc[sel.target.index, 2], Dataframe.iloc[sel.target.index, 1], Dataframe.iloc[sel.target.index, 4], Dataframe.iloc[sel.target.index, 5], Dataframe.iloc[sel.target.index, 6], Dataframe.iloc[sel.target.index, 7], Dataframe.iloc[sel.target.index, 8])))


    plt.scatter(Dataframe.iloc[Dataframe.idxmax()[2], 3], Dataframe.iloc[Dataframe.idxmax()[2], 0], marker='x', c='red', s=15)
    plt.annotate('Sharpe Max', fontsize=10, xy=(Dataframe.iloc[Dataframe.idxmax()[2], 3], Dataframe.iloc[Dataframe.idxmax()[2], 0]), xytext=(Dataframe.iloc[Dataframe.idxmax()[2], 3] -0.03 , Dataframe.iloc[Dataframe.idxmax()[2], 0]+0.15))

    plt.scatter(Dataframe.iloc[Dataframe.idxmin()[3], 3], Dataframe.iloc[Dataframe.idxmin()[3], 0], marker='x', c='red',s=15)
    plt.annotate('Volatility Min', fontsize=10, xy=(Dataframe.iloc[Dataframe.idxmin()[3], 3], Dataframe.iloc[Dataframe.idxmin()[3], 0]),
                 xytext=(Dataframe.iloc[Dataframe.idxmin()[3], 3] - 0.03, Dataframe.iloc[Dataframe.idxmin()[3], 0] + 0.15), )

    plt.scatter(Dataframe.iloc[Dataframe.idxmax()[0], 3], Dataframe.iloc[Dataframe.idxmax()[0], 0], marker='x', c='red',
                s=15)
    plt.annotate('Return Max', fontsize=10,
                 xy=(Dataframe.iloc[Dataframe.idxmax()[0], 3], Dataframe.iloc[Dataframe.idxmax()[0], 0]), xytext=(
        Dataframe.iloc[Dataframe.idxmax()[0], 3] - 0.03, Dataframe.iloc[Dataframe.idxmax()[0], 0] + 0.15))

    plt.scatter(Dataframe.iloc[Dataframe.idxmax()[1], 3], Dataframe.iloc[Dataframe.idxmax()[1], 0], marker='x', c='red',
                s=15)
    plt.annotate('MDD Min', fontsize=10,
                 xy=(Dataframe.iloc[Dataframe.idxmax()[1], 3], Dataframe.iloc[Dataframe.idxmax()[1], 0]), xytext=(
        Dataframe.iloc[Dataframe.idxmax()[1], 3] - 0.03, Dataframe.iloc[Dataframe.idxmax()[1], 0] + 0.15))

    #람다식

    plt.xlabel('Volatility')
    plt.ylabel('Return')
    plt.show()






In [3]:


def Backtesting_all(Data):
    array_bt_all = pd.DataFrame()
    t = 0
    for a in range(0, 16):
        print("Backtesting Risk Averse {} % percent..".format(int(a * 6.7)))
        a += 5
        for b in range(0, 16):
            b += 5
            if a + b > 35:
                break
            for c in range(0, 16):
                c += 5
                if a + b + c > 40:
                    break
                    
                d = 50 - a - b - c
                if d > 20:
                    break
                

                    
                 

                random_weight = np.array([a, b, c, d]) * 2

                kospi = random_weight[0]
                k10bond = random_weight[1]
                gold = random_weight[2]
                commodity = random_weight[3]


                list_for_mdd = []
                list_for_var = []

                R = None

                for i in range(0, len(Data)):
                    print( Data['gold'][i])
                    print('succgold')
                    gold = gold * (1 + Data['gold'][i])  
                    # [i] 값은 행을 의미함. (1+값)인 이유는 증감률이니깐
                    k10bond = k10bond * (1 + Data['k10bond'][i])
                    kospi = kospi * (1 + Data['kospi'][i])
                    commodity = commodity * (1 + Data['commodity'][i])

                    sum_for_mdd_var = gold + k10bond + kospi + commodity
                    list_for_mdd.append(sum_for_mdd_var)

                    if R:  # 분산 구하기 위한 짓거리..
                        r = (sum_for_mdd_var - R) / R * 100
                        list_for_var.append(r)

                    R = sum_for_mdd_var

                rate_of_return = (gold + k10bond + kospi + commodity - 100)
                Max = max(list_for_mdd)
                Min = min(list_for_mdd)
                Mdd = (Min - Max) / Max * 100

                var_portfolio = np.array(list_for_var).var()  # 분산_ 모집단 전체 대상 varp
                Volatility_standard_deviation = np.sqrt(var_portfolio)
                Sharpe_ratio = (rate_of_return/100) / Volatility_standard_deviation  # expected_return / portfolio_volatility
                    # print(var_portfolio)
                    # print(Volatility_standard_deviation)

                infor_dataframe = pd.DataFrame(
                    {'Return %': [round(rate_of_return, 4)], 'MDD %': [round(Mdd, 4)], 'Sharpe': [round(Sharpe_ratio, 4)],
                     'Volatility': [round(Volatility_standard_deviation, 4)],
                     'kospi': [random_weight[0]], 'k10bond': [random_weight[1]],
                     'gold': [random_weight[2]], 'commodity': [random_weight[3]]},
                    index=[t])

                array_bt_all = pd.concat([array_bt_all, infor_dataframe])

                t += 1
       
    return array_bt_all

Colected_Data = pd.DataFrame()
array_for_sort = pd.DataFrame()
all_array_for_sort = pd.DataFrame()

#백테스팅 날짜 선택하세요 2xxx-xx 형식 지킬것
Colected_Data = Select_Date('2003-08', '2004-02', Colected_Data) 
Colected_Data = Select_Date('2005-06', '2006-02', Colected_Data) 


print(Colected_Data)

"""
둔화 2003-02 ~ 2003-04 , 2004-04 ~ 2004-11, 2006-01 ~ 2006-07, 2008-04 ~ 2008-07, 2011-09 ~ 2012-08, 2014-02 ~ 2015-05, 2018-06 ~ 2018-12, 2019-12~ 2020-01

하강 2003-04 ~ 2003-07 , 2004-12 ~ 2005-05, 2006-08 #딱한달만  , 2012-09 ~ 2013-07, 2015-06 ~ 2016-11 , 2019-01 ~ 2019-11, 2020-06#한달만, 

회복 2003-08 ~ 2004-01 , 2005-06 ~ 2006-01, 2006-09 ~ 2006-11, 2009-09 ~ 2010-05, 2013-08 ~ 2014-01, 2016-12 ~ 2017-06, 2020-11 ~ 2021-02

상승 2004-02 ~ 2004-03 , 2006-12 ~ 2008-03. 2010-06 ~ 2011-08, 2017-07 ~ 2018-05, 2021-03 ~ 2021-06

급하강 2008-08 ~ 2009-01, 2020-02 ~ 2020-05
급회복 2009-02 ~ 2009-08, 2020-07 ~ 2020-10
"""



              gold  commodity   k10bond   kospi
date                                           
2003-08-01 -0.0223     0.0371  0.000206  0.0193
2003-08-04  0.0090    -0.0131  0.000000 -0.0120
2003-08-05  0.0011     0.0076  0.000206  0.0046
2003-08-06  0.0034    -0.0043  0.000203 -0.0193
2003-08-07  0.0048     0.0187  0.000201  0.0027
...            ...        ...       ...     ...
2006-01-25  0.0081    -0.0133  0.000211  0.0119
2006-01-26 -0.0044     0.0023  0.000209  0.0076
2006-01-27 -0.0018     0.0157  0.000210  0.0235
2006-01-30  0.0127     0.0170  0.000000  0.0000
2006-01-31  0.0088    -0.0060  0.000210  0.0110

[305 rows x 4 columns]


'\n둔화 2003-02 ~ 2003-04 , 2004-04 ~ 2004-11, 2006-01 ~ 2006-07, 2008-04 ~ 2008-07, 2011-09 ~ 2012-08, 2014-02 ~ 2015-05, 2018-06 ~ 2018-12, 2019-12~ 2020-01\n\n하강 2003-04 ~ 2003-07 , 2004-12 ~ 2005-05, 2006-08 #딱한달만  , 2012-09 ~ 2013-07, 2015-06 ~ 2016-11 , 2019-01 ~ 2019-11, 2020-06#한달만, \n\n회복 2003-08 ~ 2004-01 , 2005-06 ~ 2006-01, 2006-09 ~ 2006-11, 2009-09 ~ 2010-05, 2013-08 ~ 2014-01, 2016-12 ~ 2017-06, 2020-11 ~ 2021-02\n\n상승 2004-02 ~ 2004-03 , 2006-12 ~ 2008-03. 2010-06 ~ 2011-08, 2017-07 ~ 2018-05, 2021-03 ~ 2021-06\n\n급하강 2008-08 ~ 2009-01, 2020-02 ~ 2020-05\n급회복 2009-02 ~ 2009-08, 2020-07 ~ 2020-10\n'